In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
filenames=os.listdir('../input/agriculture-crop-images/crop_images')


In [ ]:
df=pd.DataFrame({'filename':filenames})

In [ ]:
df.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen=ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True,rotation_range=0.2,width_shift_range=0.2,height_shift_range=0.2)

In [ ]:
train_data1=data_gen.flow_from_directory('../input/agriculture-crop-images/kag2',target_size=(224,224))
train_data2=data_gen.flow_from_directory('../input/agriculture-crop-images/crop_images',target_size=(224,224))
train_data3=data_gen.flow_from_directory('../input/agriculture-crop-images/some_more_images/some_more_images',target_size=(224,224))

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
resnet=ResNet50(include_top=False,input_shape=(224,224,3))

In [ ]:
resnet.summary()

In [ ]:
for layer in resnet.layers:
    layer.trainable= False
resnet.summary()

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

In [ ]:
resnet_Flatten=Flatten()(resnet.output)
top_layer=Dense(5,activation='softmax')(resnet_Flatten)
top_layer

In [ ]:
my_resnet_model=Model(inputs=resnet.input, outputs=top_layer)
my_resnet_model.summary()

In [ ]:
my_resnet_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import tensorflow as tf

In [ ]:
# ACCURACY_THRESHOLD = 0.95

In [ ]:
# class myCallback(tf.keras.callbacks.Callback): 
#     def on_epoch_end(self, epoch, logs={}): 
#         if(logs.get('acc') > ACCURACY_THRESHOLD):   
#             print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
#             self.model.stop_training = True

In [ ]:
# callbacks = myCallback()

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
my_resnet_model.fit(train_data1, epochs=50, callbacks=my_callbacks)

In [ ]:
import cv2

In [ ]:
def get_class(img_path):
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))/255
    img=img.reshape(1,224,224,3)
    index=my_resnet_model.predict(img).argmax()
    return 'jute' if index==0 else 'maize' if index == 1 else 'rice' if index == 2 else 'sugarcane' if index==3 else 'wheat'

In [ ]:
get_class('../input/agriculture-crop-images/test_crop_image/wheatcropfield04.jpg')

In [ ]:
get_class('../input/agriculture-crop-images/test_crop_image/maize-Field-Corn.jpg')

In [ ]:
get_class('../input/agriculture-crop-images/test_crop_image/wheat-field-395545_960_720.jpg')

In [ ]:
get_class('../input/agriculture-crop-images/test_crop_image/rice-828540_1280.jpg')

In [ ]:
get_class('../input/agriculture-crop-images/test_crop_image/maizecornleaves.jfif')